## **Conociendo los datos**

Luego de haber de hecho una limpieza y aplicado algunas transformaciones sobre los datos, continuamos con algunas observaciones sobre los datos

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

import pandas as pd

In [2]:
import os
PG_JDBC_PATH = "/home/jovyan/ecobici_rides/postgresql-42.2.12.jar"
os.environ["PYSPARK_SUBMIT_ARGS"] = f"--driver-class-path {PG_JDBC_PATH} --jars {PG_JDBC_PATH} pyspark-shell"

In [3]:
spark = SparkSession.builder \
            .master("local[*]") \
            .appName("exploration") \
            .getOrCreate()

In [4]:
rides = spark.read.parquet("tables/*")

In [12]:
cols = list(filter(lambda x: x.startswith('lat') or x.startswith('long'), \
       rides.columns))

for c in cols:
    rides = rides.withColumn(c, \
                            F.col(c).cast('double'))
rides.printSchema()

root
 |-- fecha_origen_recorrido: timestamp (nullable = true)
 |-- id_estacion_origen: integer (nullable = true)
 |-- lat_estacion_origen: double (nullable = true)
 |-- long_estacion_origen: double (nullable = true)
 |-- duracion_recorrido: integer (nullable = true)
 |-- fecha_destino_recorrido: timestamp (nullable = true)
 |-- id_estacion_destino: integer (nullable = true)
 |-- lat_estacion_destino: double (nullable = true)
 |-- long_estacion_destino: double (nullable = true)



In [10]:
for _ in rides.columns:
    print(_, end="\t")
    print(rides.filter(F.isnull(_)).count())

fecha_origen_recorrido	0
id_estacion_origen	0
lat_estacion_origen	0
long_estacion_origen	0
duracion_recorrido	0
fecha_destino_recorrido	0
id_estacion_destino	0
lat_estacion_destino	0
long_estacion_destino	0


Veamos sobre los diferentes id. de las estaciones

In [11]:
for id_estacion in ["id_estacion_origen", "id_estacion_destino"]:
    rides.select(F.max(id_estacion), \
                F.min(id_estacion), \
                F.countDistinct(id_estacion)) \
        .show()

+-----------------------+-----------------------+----------------------------------+
|max(id_estacion_origen)|min(id_estacion_origen)|count(DISTINCT id_estacion_origen)|
+-----------------------+-----------------------+----------------------------------+
|                    449|                      1|                               426|
+-----------------------+-----------------------+----------------------------------+

+------------------------+------------------------+-----------------------------------+
|max(id_estacion_destino)|min(id_estacion_destino)|count(DISTINCT id_estacion_destino)|
+------------------------+------------------------+-----------------------------------+
|                     449|                       1|                                426|
+------------------------+------------------------+-----------------------------------+



Cada recorrido debería transcurrir en un mismo día. Sin embargo, observemos la siguiente particularidad

In [5]:
rides = rides.withColumn("diff_days", \
                        F.datediff("fecha_origen_recorrido", "fecha_destino_recorrido"))
rides.select("diff_days").distinct().show(5)
rides.select(F.countDistinct("diff_days"), \
            F.max("diff_days"), \
            F.min("diff_days")).show()

+---------+
|diff_days|
+---------+
|      148|
|      -35|
|     -265|
|      322|
|      -13|
+---------+
only showing top 5 rows

+-------------------------+--------------+--------------+
|count(DISTINCT diff_days)|max(diff_days)|min(diff_days)|
+-------------------------+--------------+--------------+
|                      112|           323|          -323|
+-------------------------+--------------+--------------+



In [13]:
print(f"Menor a cero: {rides.filter('diff_days < 0').count()}")
print(f"Igual a cero: {rides.filter('diff_days = 0').count()}")
print(f"Mayor a cero: {rides.filter('diff_days > 0').count()}")

Menor a cero: 178227
Igual a cero: 9968126
Mayor a cero: 51322


La nueva columna, *diff_days*, indica la cantidad de días que abarcó el recorrido.

Existen 112 valores diferentes para dicha cantidad de días.

El recorrido de máxima duración fue de 323 días, **casi un año**.

Hay recorridos de duración negativa.

Considero trabajar con los recorridos cuya duración fue de *0 días*.

In [6]:
rides = rides.filter('diff_days = 0')

Consultando los [datos](https://data.buenosaires.gob.ar/dataset/estaciones-bicicletas-publicas) del sistema de bicicletas públicas, veo que hay dos datasets sobre las estaciones de bicicletas, *inicial* y *nuevo*

In [4]:
!head -1 estaciones-bicicletas-publicas.csv

lat,long,nombre_estacion,id_estacion,capacidad,dirección_completa,direccion_nombre,direccion_altura,direccion_interseccion,barrio


In [5]:
# Sistema nuevo
cols = ['id_estacion', 'nombre_estacion', 'lat', 'long']
stations = pd.read_csv("estaciones-bicicletas-publicas.csv", usecols=cols)
stations = stations.astype({'id_estacion':'int32'})
stations.nombre_estacion = stations.nombre_estacion.str.split('-') \
                        .apply(lambda r: r[-1].strip())
print(f"Id. de estacion max. {stations.id_estacion.max()}")
print(f"Id. de estacion min. {stations.id_estacion.min()}")
print(f"Cant. de ids {stations.shape[0]}")

Id. de estacion max. 449
Id. de estacion min. 2
Cant. de ids 396


Observando los datos de las estaciones del sistema *nuevo/actual*, veo que la cantidad de estaciones es menor a la cantidad de estaciones registrados en los datasets de recorridos

In [6]:
for id_estacion in ["id_estacion_origen", "id_estacion_destino"]:
    ids_in_rides = rides.select(id_estacion).distinct().toPandas()
    other_stations = list(filter(lambda x: x not in stations['id_estacion'].values, \
                ids_in_rides[id_estacion].values))
    print(*other_stations)

133 108 34 81 103 157 140 1 16 142 178 139 37 61 35 100 136 129 97 10 113 160 125 145 109 147 198 123 67 18
133 108 34 81 103 157 140 1 16 142 178 139 37 61 35 100 136 129 97 10 113 160 125 145 109 147 198 123 67 18


Verificamos que los id. listados anteriormente se encuentran registrados en los datos del sistema *inicial*

In [7]:
!head -1 estaciones_sistema_viejo.csv
cols = ['id_estacion', 'nombre_estacion', 'lat_estacion', 'long_estacion']
old_stations = pd.read_csv("estaciones_sistema_viejo.csv", usecols=cols)
old_stations.dropna(subset=['id_estacion'], inplace=True)
old_stations = old_stations.astype({'id_estacion': 'int32'})
old_stations = old_stations.set_index('id_estacion')

id_estacion,nombre_estacion,long_estacion,lat_estacion,domicilio_estacion,tipo_estacion,observaciones,horario_estacion


In [8]:
old_stations.loc[other_stations]

,nombre_estacion,long_estacion,lat_estacion
id_estacion,,,
133,Nicaragua,-58.434383,-34.581040
108,Usina del Arte,-58.356257,-34.628781
34,Galerías Pacífico,-58.374157,-34.599579
81,Balcarce,-58.370879,-34.615847
103,Malba,-58.403533,-34.577415
157,Senillosa y Zuviría,-58.427952,-34.628309
140,Las Casas,-58.415775,-34.635148
1,Facultad de Derecho,-58.392452,-34.583133
16,Legislatura,-58.374717,-34.609959


In [9]:
stations = stations.loc[:, ['id_estacion', 'nombre_estacion', 'lat', 'long']]
stations

,id_estacion,nombre_estacion,lat,long
0,2,Retiro I,-34.592423,-58.374715
1,3,ADUANA,-34.611032,-58.368260
2,4,Plaza Roma,-34.601726,-58.368763
3,5,Plaza Italia,-34.580550,-58.420954
4,6,Parque Lezama,-34.628526,-58.369758
...,...,...,...,...
391,448,Barrio 31,-34.583250,-58.378731
392,76,Ayacucho,-34.607408,-58.395055
393,328,Plaza Houssay,-34.598549,-58.397354
394,449,San Jose de Flores,-34.628937,-58.463661


In [10]:
others = old_stations.loc[other_stations] \
                        .reset_index() \
                        .loc[:,['id_estacion', 'nombre_estacion', 'lat_estacion', 'long_estacion']] \
                        .rename(columns={'lat_estacion': 'lat', 'long_estacion': 'long'})
others.head()

,id_estacion,nombre_estacion,lat,long
0,133,Nicaragua,-34.581040,-58.434383
1,108,Usina del Arte,-34.628781,-58.356257
2,34,Galerías Pacífico,-34.599579,-58.374157
3,81,Balcarce,-34.615847,-58.370879
4,103,Malba,-34.577415,-58.403533


In [11]:
all_stations = stations.append(others, ignore_index=True)
all_stations

,id_estacion,nombre_estacion,lat,long
0,2,Retiro I,-34.592423,-58.374715
1,3,ADUANA,-34.611032,-58.368260
2,4,Plaza Roma,-34.601726,-58.368763
3,5,Plaza Italia,-34.580550,-58.420954
4,6,Parque Lezama,-34.628526,-58.369758
...,...,...,...,...
421,147,Constitución I,-34.627455,-58.379105
422,198,Hospital Sardá,-34.634380,-58.404138
423,123,Armenia,-34.585424,-58.421044
424,67,Senillosa,-34.638501,-58.425477


In [12]:
all_stations.to_csv('all_stations.csv', index=False)

In [13]:
!head all_stations.csv

id_estacion,nombre_estacion,lat,long
2,Retiro I,-34.59242329999999,-58.37471510000001
3,ADUANA,-34.611032,-58.3682604
4,Plaza Roma,-34.6017255,-58.368762800000006
5,Plaza Italia,-34.580549700000006,-58.420954200000004
6,Parque Lezama,-34.628526,-58.369758
7,OBELISCO,-34.606497999999995,-58.381098
8,Congreso,-34.6094218,-58.3893364
9,Parque Las Heras,-34.585443,-58.407741
11,Tribunales,-34.601176200000005,-58.3850791


In [16]:
!tail all_stations.csv

113,Guatemala,-34.5856694443182,-58.42520465764979
160,Godoy Cruz y Libertador,-34.5729623675618,-58.42021025469279
125,Oro,-34.5834801072487,-58.4282171673392
145,Ravignani,-34.5804694653892,-58.4378366120078
109,Lerma,-34.5966937447772,-58.4295906822899
147,Constitución I,-34.6274554061156,-58.37910486128021
198,Hospital Sardá,-34.6343800087707,-58.4041380643401
123,Armenia,-34.5854242215495,-58.421044153603795
67,Senillosa,-34.6385009158718,-58.425477470589
18,Independencia,-34.6173731525033,-58.3804806550249


In [8]:
rides = rides.drop("diff_days")

In [ ]:
connection_string = "jdbc:postgresql://127.0.0.1/ecobici"
tablename = "public.rides"
connection_details = {
    "user": "<uname>",
    "password": "<pswd>",
    "driver": "org.postgresql.Driver"
}

In [ ]:
rides.write.jdbc(connection_string, tablename, mode='append', properties=connection_details)